In [17]:
from datetime import datetime, timedelta
import mysql.connector
import pandas as pd
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType
from textblob import TextBlob
from pyspark.sql.functions import udf

TOPIC='twitter'
DATABASE = 'twitter'
USERNAME = 'root'
PASSWORD = '123'

print("Connecting to the database")
try:
    connection = mysql.connector.connect(host='172.16.0.5', database=DATABASE, user=USERNAME, password=PASSWORD)
except Exception:
    print("Could not connect to database. Please check credentials")
else:
    print("Connected to database")
cursor = connection.cursor()


spark = SparkSession\
    .builder\
    .master("spark://master:7077")\
    .appName("transform")\
    .enableHiveSupport()\
    .getOrCreate()

def sentiment_score(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1
udf_sentiment_score = udf(sentiment_score, IntegerType())

def who(text):
    text_lower = text.lower()
    print(text_lower)
    ronaldo = 0
    messi = 0
    #ronaldo, #messi
    if(('ronaldo') in text_lower) or (('cr7') in text_lower):
        ronaldo = 1
    if(('messi') in text_lower) or(('m10') in text_lower):
        messi = 1
    if(ronaldo == messi):
        if(ronaldo == 1):
            return 'both'
        else:
            return 'another'
    elif(messi==1):
        return 'messi'
    else:
        return 'ronaldo'
udf_who = udf(who, StringType())

def source(text):
    return text.split(">")[1].split("<")[0]
udf_source = udf(source, StringType())

date_time = datetime.today().replace(second = 0) + timedelta(hours = 7) - timedelta(minutes = 5)

now = (date_time).strftime("%Y-%m-%d %H:%M:%S") 
now_add_1 = (date_time + timedelta(minutes = 6)).strftime("%Y-%m-%d %H:%M:%S")

sql = "select create_at, link, text, source from tweet where create_at > " + "'" + now + "'" + " and create_at <= " + "'" + now_add_1 + "'"
pdf = pd.read_sql(sql, con=connection)
schema = StructType([ \
    StructField("create_at",StringType(),True), \
    StructField("link",StringType(),True), \
    StructField("text",StringType(),True), \
    StructField("source",StringType(),True)
                    ])
df = spark.createDataFrame(pdf,schema=schema)

#df.show()
df_sentiment_score = df.withColumn('score', udf_sentiment_score('text'))

#df_sentiment_score.show()

df_who = df_sentiment_score.withColumn('who', udf_who('text'))

#df_who.show()

df_who_filter = df_who.filter("who like 'messi' or who like 'ronaldo'")

#df_who_filter.show()

df_source = df_who_filter.withColumn('media_source', udf_source('source'))

#df_source.show()

path = (date_time).strftime("%Y/%m/%d/%H/%M")

#df_source.write.parquet("hdfs://master:9000/data/" + path)

df_source = df_source.withColumn("create_at", (df_source.create_at.cast("timestamp")))

df_source.show()

df_source.write.mode("append").saveAsTable("twitter.minutes")

spark.stop()


Connecting to the database
Connected to database


21/11/29 12:22:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/11/29 12:22:03 WARN SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


+---------+----+----+------+
|create_at|link|text|source|
+---------+----+----+------+
+---------+----+----+------+



+---------+----+----+------+-----+
|create_at|link|text|source|score|
+---------+----+----+------+-----+
+---------+----+----+------+-----+



+---------+----+----+------+-----+---+
|create_at|link|text|source|score|who|
+---------+----+----+------+-----+---+
+---------+----+----+------+-----+---+



+---------+----+----+------+-----+---+
|create_at|link|text|source|score|who|
+---------+----+----+------+-----+---+
+---------+----+----+------+-----+---+

+---------+----+----+------+-----+---+------------+
|create_at|link|text|source|score|who|media_source|
+---------+----+----+------+-----+---+------------+
+---------+----+----+------+-----+---+------------+

+---------+----+----+------+-----+---+------------+
|create_at|link|text|source|score|who|media_source|
+---------+----+----+------+-----+---+------------+
+---------+----+----+------+-----+---+------------+



In [16]:

spark.stop()

In [19]:
spark.stop()

In [3]:
connection = mysql.connector.connect(host='172.16.0.5', database=DATABASE, user=USERNAME, password=PASSWORD)

In [ ]:
apt install -y python3-mysql.connector
pip3 install pandas
pip3 install findspark
pip3 install textblob


In [16]:
from datetime import datetime
datetime.today().replace(second=0)

datetime.datetime(2021, 11, 23, 0, 46, 0, 830338)

In [93]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
#spark = SparkSession.builder.master("spark://master:7077").getOrCreate()

spark = SparkSession\
    .builder\
    .master("spark://master:7077")\
    .appName("x")\
    .config("spark.jars", "./mysql-connector-java-8.0.27.jar")\
    .config("spark.submit.deployMode", "cluster")\
    .config("spark.driver.extraClassPath", "./mysql-connector-java-8.0.27.jar")\
    .config("spark.jars.packages", "./mysql-connector-java-8.0.27.jar")\
    .getOrCreate()
    #.config("spark.driver.extraClassPath", "/home/tuhin/mysql.jar")\

# SUBMIT_ARGS = "--packages mysql:mysql-connector-java:5.1.39 pyspark-shell"
# os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS
    
# dataframe_mysql = spark.read.format("jdbc").options(
#     url="jdbc:mysql://loclahost:3306/twitter",
#     driver = "com.mysql.jdbc.Driver",
#     dbtable = "tweet",
#     user="root",
#     password="123").load()

# dataframe_mysql.show()    

# dataframe_mysql = spark.read\
#     .format("jdbc")\
#     .option("url", "jdbc:mysql://172.16.0.5/twitter")\
#     .option("driver", "com.mysql.jdbc.Driver")\
#     .option("dbtable", "tweet").option("user", "root")\
#     .option("password", "123").load()

#spark.stop()

In [197]:
spark.stop()

In [92]:
dataframe_mysql = spark.read.format("jdbc").options(
    url="jdbc:mysql://172.16.0.5:3306/twitter",
    driver = "com.mysql.jdbc.Driver",
    dbtable = "tweet",
    user="root",
    password="123").load()

Py4JJavaError: An error occurred while calling o1482.load.
: java.lang.ClassNotFoundException: com.mysql.jdbc.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:45)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:99)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:99)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:99)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:35)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:32)
	at org.apache.spark.sql.exe